# BlueShift Store Data
    This notebook will generate a dummy data set of a made up online store called BlueShift.
    The data set will consist of customers, products, sellers, and orders tables.

In [1]:
from faker import Faker
from faker_commerce import Provider
from random import choice
import datetime
import pandas as pd
import sqlalchemy, pyodbc, os

In [2]:
fake = Faker(['en_US'])

### The customers table will have 5,000 rows with colomns:
    ID [Primary Key]
    First_Name - Customer's first name
    Last_Name - Customer's last name
    Gender - Customer's gender
    Address - Customer's address
    E-mail - Customer's e-mail address
    Primary_Payment_Method - Payment method customer uses most often

In [3]:
customer_ids = {'Header': 'ID', 'Values': []}
customer_first_names = {'Header': 'First_Name', 'Values': []}
customer_last_names = {'Header': 'Last_Name', 'Values': []}
genders = {'Header': 'Gender', 'Values': []}
street_addresses = {'Header': 'Street_Address', 'Values': []}
cities = {'Header': 'City', 'Values': []}
customer_states = {'Header': 'State', 'Values': []}
customer_zipcodes = {'Header': 'Postal', 'Values': []}
customer_emails = {'Header': 'Email', 'Values': []}
pay_methods = {'Header': 'Primary_Payment_Methods', 'Values': []}

for _ in range(5_000):
    customer_ids['Values'].append(fake.unique.random_int(min=10_000, max=99_999))
    customer_first_names['Values'].append(fake.first_name())
    customer_last_names['Values'].append(fake.last_name())
    genders['Values'].append(choice(['Male', 'Female']))
    street_addresses['Values'].append(fake.street_address())
    cities['Values'].append(fake.city())
    customer_states['Values'].append(fake.state())
    customer_zipcodes['Values'].append(fake.postcode())
    customer_emails['Values'].append(fake.email())
    pay_methods['Values'].append(choice(['Credit Card', 'PayPal', 'E-Wallet']))

### The products table will have 150 rows with columns:
    ID [Primary Key]
    Product_Name - Name of product
    Unit_Price - Price of individual product

In [4]:
product_ids = {'Header': 'ID', 'Values': []}
product_names = {'Header': 'Name', 'Values': []}
unit_prices = {'Header': 'Unit_Price', 'Values': []}

fake.add_provider(Provider)

for _ in range(150):
    product_ids['Values'].append(fake.unique.random_int(min=1_000, max=9_999))
    product_names['Values'].append(fake.unique.ecommerce_name())
    unit_prices['Values'].append(float(fake.random_int(min=5, max=999))*.99)

### The sellers table will have 988 rows with columns:
    ID [Primary Key]
    First_Name - Seller's first name
    Last_Name - Seller's last name
    State - State where seller lives
    Five_Stars - Number of five-star reviews
    Four_Stars - Number of four-star reviews
    Three_Stars - Number of three-star reviews
    Two_Stars - Number of two-star reviews
    One_Stars - Number of one-star reviews

In [5]:
seller_ids = {'Header': 'ID', 'Values': []}
seller_first_names = {'Header': 'First_Name', 'Values': []}
seller_last_names = {'Header': 'Last_Name', 'Values': []}
seller_states = {'Header': 'State', 'Values': []}
seller_five_stars = {'Header': 'Five_Star_Reviews', 'Values': []}
seller_four_stars = {'Header': 'Four_Star_Reviews', 'Values': []}
seller_three_stars = {'Header': 'Three_Star_Reviews', 'Values': []}
seller_two_stars = {'Header': 'Two_Star_Reviews', 'Values': []}
seller_one_stars = {'Header': 'One_Star_Reviews', 'Values': []}

for _ in range(988):
    seller_ids['Values'].append(fake.unique.random_int(min=100_000, max=899_999))
    seller_first_names['Values'].append(fake.first_name())
    seller_last_names['Values'].append(fake.first_name())
    seller_states['Values'].append(fake.state())
    seller_five_stars['Values'].append(fake.random_int(min=1, max=100))
    seller_four_stars['Values'].append(fake.random_int(min=1, max=100))
    seller_three_stars['Values'].append(fake.random_int(min=1, max=100))
    seller_two_stars['Values'].append(fake.random_int(min=1, max=100))
    seller_one_stars['Values'].append(fake.random_int(min=1, max=100))

### The orders table will have 45000 to 55000 rows with columns:
    ID [Primary Key]
    Customer_ID [Foreign Key]
    Seller_ID [Foreign Key]
    Number_of_Items - Number of items in the order (can include unique and/or duplicate items)
    Order_Date - Date the order was placed, the years used are 2021 and 2022
    Order_Time - Time the order was placed
    Estimated_Delivery_Date - Date the order was estimated to arrive to the customer
    Date_Delivered - Date the order was actually delivered

In [6]:
order_ids = {'Header': 'ID', 'Values': []}
o_customer_ids = {'Header': 'Customer_ID', 'Values': []}
o_seller_ids = {'Header': 'Seller_ID', 'Values': []}
number_of_items = {'Header': 'Number_of_Items', 'Values': []}
order_dates = {'Header': 'Order_Date', 'Values': []}
order_times = {'Header': 'Order_Time', 'Values': []}
est_deliv_dates = {'Header': 'Estimated_Delivery_Date', 'Values': []}
date_delivered = {'Header': 'Date_delivered', 'Values': []}
order_item_count_dict = {}

num_rows = fake.random_int(min=45_000, max=55_000)
for _ in range(num_rows):
    o_id = fake.unique.random_int(min=1_000_000, max=3_499_999)
    order_ids['Values'].append(o_id)
    o_customer_ids['Values'].append(choice(customer_ids['Values']))
    o_seller_ids['Values'].append(choice(seller_ids['Values']))
    num = fake.random_int(min=1, max=8)
    number_of_items['Values'].append(num)
    
    date = fake.date_between(datetime.date(2021, 1, 1), datetime.date(2022, 12, 31))
    date_str = date.strftime('%Y-%m-%d')
    order_dates['Values'].append(date_str)
    
    order_times['Values'].append(fake.time())
    
    est_delta = datetime.timedelta(days=fake.random_int(min=3, max=7))
    est_delta_str = fake.date_between(date, date + est_delta).strftime('%Y-%m-%d')
    est_deliv_dates['Values'].append(est_delta_str)
    
    real_delta = datetime.timedelta(days=fake.random_int(min=3, max=9))
    real_delta_str = fake.date_between(date, date + real_delta).strftime('%Y-%m-%d')
    date_delivered['Values'].append(real_delta_str)
    
    # Find how many items each order has
    order_item_count_dict[o_id] = num

### The product_orders table will have [___] rows with columns:

    Order_ID [Foreign Key]
    Product_ID [Foreign Key]

In [7]:
po_order_ids = {'Header': 'Order_ID', 'Values': []}
po_product_ids = {'Header': 'Product_ID', 'Values': []}

for order_id_index in range(len(order_ids['Values'])):
    num = number_of_items['Values'][order_id_index]
    for _ in range(num):
        po_order_ids['Values'].append(order_ids['Values'][order_id_index])
        po_product_ids['Values'].append(choice(product_ids['Values']))

In [8]:
# Check that po_order_ids and po_product_ids have the same number of elements
print(len(po_order_ids['Values']))
print(len(po_product_ids['Values']))

203826
203826


### Write the customers table to a .csv file

In [9]:
customer_df = pd.DataFrame()
customer_df.insert(loc=0, column=customer_ids['Header'], value=customer_ids['Values'])
customer_df.insert(loc=1, column=customer_first_names['Header'], value=customer_first_names['Values'])
customer_df.insert(loc=2, column=customer_last_names['Header'], value=customer_last_names['Values'])
customer_df.insert(loc=3, column=genders['Header'], value=genders['Values'])
customer_df.insert(loc=4, column=street_addresses['Header'], value=street_addresses['Values'])
customer_df.insert(loc=5, column=cities['Header'], value=cities['Values'])
customer_df.insert(loc=6, column=customer_states['Header'], value=customer_states['Values'])
customer_df.insert(loc=7, column=customer_zipcodes['Header'], value=customer_zipcodes['Values'])
customer_df.insert(loc=8, column=customer_emails['Header'], value=customer_emails['Values'])
customer_df.insert(loc=9, column=pay_methods['Header'], value=pay_methods['Values'])

filename = 'csv_files\\customer_fake_data.csv'
customer_df.to_csv(path_or_buf=filename, mode='w', index=False)

### Write the products table to a .csv file

In [10]:
product_df = pd.DataFrame()
product_df.insert(loc=0, column=product_ids['Header'], value=product_ids['Values'])
product_df.insert(loc=1, column=product_names['Header'], value=product_names['Values'])
product_df.insert(loc=2, column=unit_prices['Header'], value=unit_prices['Values'])

filename = 'csv_files\\product_fake_data.csv'
product_df.to_csv(path_or_buf=filename, mode='w', index=False)

### Write the sellers table to a .csv file

In [11]:
seller_df = pd.DataFrame()
seller_df.insert(loc=0, column=seller_ids['Header'], value=seller_ids['Values'])
seller_df.insert(loc=1, column=seller_first_names['Header'], value=seller_first_names['Values'])
seller_df.insert(loc=2, column=seller_last_names['Header'], value=seller_last_names['Values'])
seller_df.insert(loc=3, column=seller_states['Header'], value=seller_states['Values'])
seller_df.insert(loc=4, column=seller_five_stars['Header'], value=seller_five_stars['Values'])
seller_df.insert(loc=5, column=seller_four_stars['Header'], value=seller_four_stars['Values'])
seller_df.insert(loc=6, column=seller_three_stars['Header'], value=seller_three_stars['Values'])
seller_df.insert(loc=7, column=seller_two_stars['Header'], value=seller_two_stars['Values'])
seller_df.insert(loc=8, column=seller_one_stars['Header'], value=seller_one_stars['Values'])

filename = 'csv_files\\seller_fake_data.csv'
seller_df.to_csv(path_or_buf=filename, mode='w', index=False)

### Write the orders table to a .csv file

In [12]:
order_df = pd.DataFrame()
order_df.insert(loc=0, column=order_ids['Header'], value=order_ids['Values'])
order_df.insert(loc=1, column=o_customer_ids['Header'], value=o_customer_ids['Values'])
order_df.insert(loc=2, column=o_seller_ids['Header'], value=o_seller_ids['Values'])
order_df.insert(loc=3, column=number_of_items['Header'], value=number_of_items['Values'])
order_df.insert(loc=4, column=order_dates['Header'], value=order_dates['Values'])
order_df.insert(loc=5, column=order_times['Header'], value=order_times['Values'])
order_df.insert(loc=6, column=est_deliv_dates['Header'], value=est_deliv_dates['Values'])
order_df.insert(loc=7, column=date_delivered['Header'], value=date_delivered['Values'])

filename = 'csv_files\\order_fake_data.csv'
order_df.to_csv(path_or_buf=filename, mode='w', index=False)

### Write the product_orders table to a .csv file

In [13]:
product_order_df = pd.DataFrame()
product_order_df.insert(loc=0, column=po_order_ids['Header'], value=po_order_ids['Values'])
product_order_df.insert(loc=1, column=po_product_ids['Header'], value=po_product_ids['Values'])

filename = 'csv_files\\product_order_fake_data.csv'
product_order_df.to_csv(path_or_buf=filename, mode='w', index=False)

## Import files to SQL Server

In [14]:
# Find which ODBC driver to use in the connection string
print(pyodbc.drivers())

['SQL Server', 'SQL Server Native Client RDA 11.0', 'ODBC Driver 17 for SQL Server']


In [15]:
# Create connection to SQL Server database
connection_string = f'mssql+pyodbc://{os.environ.get("SQL_AUTH")}/Synthetic_e-commerce_store?driver=ODBC+Driver+17+for+SQL+Server'
conn = sqlalchemy.create_engine(connection_string)
customer_df.to_sql('customers', con=conn, if_exists='append', index=False)
product_df.to_sql('products', con=conn, if_exists='append', index=False)
seller_df.to_sql('sellers', con=conn, if_exists='append', index=False)
order_df.to_sql('orders', con=conn, if_exists='append', index=False)
product_order_df.to_sql('product_orders', con=conn, if_exists='append',index=False)

826